# Instruct Validate Repair
This Jupyter notebook runs the instruct-validate-repair pattern on Colab.

## Install Ollama

Before we get started with Mellea, we download, install and serve ollama. We define set_css to wrap Colab output.

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh > /dev/null
!nohup ollama serve >/dev/null 2>&1 &

from IPython.display import HTML, display


def set_css():
    display(HTML("\n<style>\n pre{\n white-space: pre-wrap;\n}\n</style>\n"))


get_ipython().events.register("pre_run_cell", set_css)

## Install Mellea
We run `uv pip install mellea` to install Mellea.

In [ ]:
!uv pip install mellea -q

## Import Mellea and Specify Requirements
We create 3 requirements:
- First requirement (r1) will be validated by LLM-as-a-judge on the output of the instruction. This is the default behavior.
- Second requirement (r2) uses a function that takes the output of a sampling step and returns a boolean value indicating successful or unsuccessful validation. While the validation_fn parameter requires to run validation on the full session context, Mellea provides a wrapper for simpler validation functions (simple_validate(fn: Callable[[str], bool])) that take the output string and return a boolean as seen in this case.
- Third requirement is a check(). Checks are only used for validation, not for generation. Checks aim to avoid the "do not think about B" effect that often primes models (and humans) to do the opposite and "think" about B.

In [ ]:
import mellea
from mellea.stdlib.requirements import check, req, simple_validate
from mellea.stdlib.sampling import RejectionSamplingStrategy

requirements = [
    req("The email should have a salutation"),  # == r1
    req(
        "Use only lower-case letters",
        validation_fn=simple_validate(lambda x: x.lower() == x),
    ),  # == r2
    check("Do not mention purple elephants."),  # == r3
]

## Create Email Function with Instruct Validate Repair
We bring it together with a function using the instruct-validate-repair pattern.

In [ ]:
def write_email(m: mellea.MelleaSession, name: str, notes: str) -> str:
    email_candidate = m.instruct(
        "Write an email to {{name}} using the notes following: {{notes}}.",
        requirements=requirements,
        strategy=RejectionSamplingStrategy(loop_budget=5),
        user_variables={"name": name, "notes": notes},
        return_sampling_results=True,
    )
    if email_candidate.success:
        return str(email_candidate.result)
    else:
        return email_candidate.sample_generations[0].value

We initialize a backend and request an email to be written.

In [ ]:
m = mellea.start_session()
print(
    write_email(
        m,
        "Olivia",
        "Olivia helped the lab over the last few weeks by organizing intern events, advertising the speaker series, and handling issues with snack delivery.",
    )
)